In [1]:
!pip install -U streamlit transformers langchain pdfminer.six chromadb pypdf sentence-transformers accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.6/272.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.1 MB/s eta 0:00:00

In [2]:
import os
import torch
from transformers import pipeline
from langchain.document_loaders import PyPDFLoader
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

In [3]:
## Load the pdf file using Langchain
pdf_file = '/content/Unit 5 3D Objects Representation.pdf'
loader = PyPDFLoader(pdf_file)
docs = loader.load_and_split()

In [4]:
print(docs[0].page_content)

3D OBJECTS 
REPRESENTATION
U N I T  5


In [5]:
embeddings = SentenceTransformerEmbeddings()

db = Chroma.from_documents(docs, embeddings,persist_directory='.')
db.persist()

In [6]:
checkpoint = 'MBZUAI/LaMini-T5-738M'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [7]:
pipe = pipeline(
    'text2text-generation',
    model = base_model,
    tokenizer=tokenizer,
    max_length=512,
    do_sample=True,
    top_p=0.95,
    temperature=0.3
    )

llm = HuggingFacePipeline(pipeline=pipe)
retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm,chain_type='stuff',
    retriever=retriever
)
generated_text = qa('What is 3D object representation?')
answer = generated_text['result']
answer

'3D object representation is a way of representing objects in 3D space by partitioning the spatial region containing them into smaller, non-overlapping, contiguous solids. It is helpful for describing interior properties of objects, such as trees and flowers, and can be divided into two categories: boundary representations (B-reps) and exterior representations (B-reps).'

# Streamlit app

In [8]:
%%writefile app.py

import os
import streamlit as st
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

def main():
    # Ask user for PDF file
    pdf_file = st.file_uploader("Upload a PDF file", type="pdf")

    if pdf_file is not None:
        # Load PDF file using Langchain
        loader = PyPDFLoader(pdf_file)
        docs = loader.load_and_split()
        embeddings = SentenceTransformerEmbeddings()

        # Create Chroma database from documents
        db = Chroma.from_documents(docs, embeddings, persist_directory='.')
        db.persist()

        # Load T5 model and tokenizer
        checkpoint = 'MBZUAI/LaMini-T5-738M'
        tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

        # Create text generation pipeline
        pipe = pipeline(
            'text2text-generation',
            model=base_model,
            tokenizer=tokenizer,
            max_length=512,
            do_sample=True,
            top_p=0.95,
            temperature=0.3
        )

        llm = HuggingFacePipeline(pipeline=pipe)
        retriever = db.as_retriever()
        qa = RetrievalQA.from_chain_type(
            llm, chain_type='stuff',
            retriever=retriever
        )

        # Ask user for question and generate answer
        question = st.text_input("Enter your question")

        if st.button("Generate Answer"):
            generated_text = qa(question)
            answer = generated_text['result']
            st.write(answer)


if __name__ == "__main__":
    main()

Writing app.py
